# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **weather phenomena** (see below) for the region of **Moscow, Moscow, Russia**, or **Russia** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Moscow, Moscow, Russia** to Ann Arbor, USA. In that case at least one source file must be about **Moscow, Moscow, Russia**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Moscow, Moscow, Russia** and **weather phenomena**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **weather phenomena**?  For this category you might want to consider seasonal changes, natural disasters, or historical trends.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [5]:
#https://rp5.ru/Weather_archive_in_Moscow
%matplotlib notebook

import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

plt.style.use('seaborn-colorblind')

df = pd.read_csv('Moscow.csv', skiprows = 6,sep = ';', index_col = False, 
                 usecols=['Local time in Moscow','T','Po','U','Ff','N','RRR','sss'])

df.columns = ['LocalTime','Temperature','Pressure','Humidity','Wind','CloudCover','Precipitation','SnowDepth']
df.set_index('LocalTime', inplace = True, drop = False)

###Cleaning the data###

#Precipitation
df['Precipitation'].fillna(0, inplace = True)
condition = (df['Precipitation'] == 'No precipitation') | (df['Precipitation'] == 'Trace of precipitation')
df['Precipitation'] = np.where(condition, 0, df['Precipitation'])
df['Precipitation'] = df['Precipitation'].astype(float)

#CloudCover 
df['CloudCover'] = df['CloudCover'].apply(lambda x: str(x).replace('%.',''))
df.loc[df['CloudCover'] == '70 – 80','CloudCover'] = '75'
df.loc[df['CloudCover'] == '90  or more, but not 100%','CloudCover'] = '90'
df.loc[df['CloudCover'] == 'no clouds','CloudCover'] = '0'
df.loc[df['CloudCover'] == '10%','CloudCover'] = '10'
df.loc[df['CloudCover'] == '10%  or less, but not 0','CloudCover'] = '10'
df.loc[df['CloudCover'] == '20–30','CloudCover'] = '25'
df.loc[df['CloudCover'] == 'nan','CloudCover'] = '0'
df.loc[df['CloudCover'] == 'Sky obscured by fog and/or other meteorological phenomena.','CloudCover'] = 100
df['CloudCover'] = df['CloudCover'].astype(int)

#SnowDepth
df.loc[df['SnowDepth'] == 'less than 0.5','SnowDepth'] = '0'
df['SnowDepth'].fillna(method='ffill', inplace = True)
df['SnowDepth'].fillna(0, inplace = True)
df['SnowDepth'] = df['SnowDepth'].astype(float)

df['Pressure'].fillna(method='ffill', inplace = True)
df['Humidity'].fillna(method='ffill', inplace = True)

df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M')

df['Year'] = df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['Hour'] = df.index.hour

ThunderDate=pd.to_datetime('2017-05-29 15:00')


In [ ]:
plt.figure(figsize=(10,6))

nightData = (df[(df['Hour'] == 0)]
             .set_index(['Year','Month'])
             .groupby(level=['Year','Month']) 
             .agg({'LocalTime' : np.min, 'Temperature' : np.mean}))

dayData = (df[(df['Hour'] == 12)]
             .set_index(['Year','Month'])
             .groupby(level=['Year','Month']) 
             .agg({'LocalTime' : np.min, 'Temperature' : np.mean}))

date_range = nightData.LocalTime.unique()

plt.plot(date_range, nightData.Temperature, '-', alpha=0.7,  label='Precipitation', c='b', linewidth=0.5);
plt.plot(date_range, dayData.Temperature, '-', alpha=0.7,  label='Precipitation', c='r', linewidth=0.5);

plt.legend();

In [ ]:
plt.figure(figsize=(10,6))

temp_df = (df[df['Year'] == 2017].set_index(['Day','Month'])
             .groupby(level=['Day','Month']) 
             .agg({'LocalTime' : np.min, 'Temperature' : np.mean}))

date_range = temp_df['LocalTime'].unique()
    
for i in df.Year.unique():
    temp_df = df[(df['Year'] == i)].set_index(['Month','Day'])
    if (2,29) in temp_df.index:
        temp_df.drop((2,29), inplace=True)
               
    temp_df = (temp_df.groupby(level=['Month','Day']).agg({'LocalTime' : np.min, 'Temperature' : np.mean}))   
    
    plt.plot(date_range, temp_df.Temperature, '-', alpha=0.7,  label='Mean T '+str(i), c='b', linewidth=0.5);

plt.legend();


In [ ]:
plt.figure(figsize=(10,6))

temp_df = (df[df['Year'] == 2017].set_index(['Month'])
             .groupby(level=['Month']) 
             .agg({'LocalTime' : np.min, 'Temperature' : np.mean}))

date_range = temp_df['LocalTime'].unique()
    
for i in df.Year.unique():
    temp_df = df[(df['Year'] == i)].set_index(['Month'])
    if (2,29) in temp_df.index:
        temp_df.drop((2,29), inplace=True)
               
    temp_df = (temp_df.groupby(level=['Month']).agg({'LocalTime' : np.min, 'Temperature' : np.mean}))   
    
    plt.plot(date_range, temp_df.Temperature, '-', alpha=0.7,  label='Mean T '+str(i), c='b', linewidth=0.5);

plt.legend();

In [8]:
df.set_index(['Year','Month']).groupby(level=['Year','Month']).sum()['Precipitation']

Year  Month
2014  1         81.3
      2         40.2
      3         35.9
      4         45.4
      5        140.2
      6        146.0
      7          8.5
      8        166.3
      9         76.7
      10        69.1
      11        41.1
      12       126.3
2015  1        117.7
      2         78.0
      3         28.7
      4         82.3
      5        241.0
      6        187.0
      7        237.4
      8         27.5
      9        174.0
      10        40.6
      11        82.8
      12       125.2
2016  1        153.0
      2        125.4
      3         96.5
      4         67.8
      5        125.4
      6        122.3
      7        242.2
      8        332.0
      9        119.8
      10        98.3
      11       182.2
      12        93.6
2017  1         88.2
      2         71.0
      3        109.5
      4        154.2
      5        162.1
      6        266.4
      7        221.2
      8        135.9
      9         75.8
      10       182.8
      11        93.5
 